In [ ]:
from google.colab import drive 
drive.mount ('/content/drive')

1.   Imports utiles



In [ ]:
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.feature_selection import SelectPercentile
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import GridSearchCV,cross_val_score
from albumentations import MotionBlur
from sklearn.preprocessing import StandardScaler 
import os
from scipy import stats
import pandas as pd
import seaborn as sns; sns.set()
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from tqdm import tqdm

2.   Importer le modèle pré-entrainé



In [ ]:
vgg19=VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

3.   Le chemin d'accès à la base de données



In [ ]:
train_path="/content/drive/MyDrive/tomato_train"
test_path="/content/drive/MyDrive/tomato_test"

4.   Extraction des variables

In [ ]:
def blur_fct(img):
 transform=MotionBlur(p=0.2)
 return(transform(image=img)['image'])

In [ ]:
#file generator
class FG : 
  train_dir = os.path.join(train_path)
  test_dir = os.path.join(test_path)
  datagenerator = ImageDataGenerator(preprocessing_function=blur_fct,rotation_range=90)
  batch_size = 20

In [ ]:
target_size=(224,224)
def feature_extractor(path, nb_images,base_model):
    features = np.empty(shape=(nb_images,base_model.output_shape[1],
                               base_model.output_shape[2],base_model.output_shape[3]))
    labels = np.empty(shape=(nb_images, 10))
    generator =FG.datagenerator.flow_from_directory(path,target_size=target_size,
                                    batch_size=FG.batch_size,class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in tqdm(generator):
      preprocessed_batch=preprocess_input(inputs_batch)
    #will convert the input images from RGB to BGR, 
    #then will zero-center each color channel with respect to the ImageNet dataset, 
    #without scaling.
      features_batch = base_model.predict(preprocessed_batch)
      features[i * FG.batch_size : (i + 1) * FG.batch_size] = features_batch
      labels[i * FG.batch_size : (i + 1) * FG.batch_size] = labels_batch
      i += 1
      if i * FG.batch_size==nb_images:
          break
    return features,labels

In [ ]:
train_features,train_labels= feature_extractor(FG.train_dir,10000,vgg19)

In [ ]:
test_features,test_labels=feature_extractor(FG.test_dir,1000,vgg19)

5.   Les noms de classes 



In [ ]:
generator = FG.datagenerator.flow_from_directory(train_path,target_size=(224,224),
                                        batch_size=20,class_mode='categorical')
class_indices=generator.class_indices
class_names={class_indices[i]:i for i in class_indices.keys()}
class_names

6.   Manipulations des nouvelles bases de données à l'aide de Pandas



a) Stockage des données dans des fichiers csv

In [ ]:
n_variables=vgg19.output_shape[1]*vgg19.output_shape[2]*vgg19.output_shape[3]
train_features=np.reshape(train_features,(10000,n_variables))
df1=pd.DataFrame(train_features, columns = ['feature {}'.format(i) for i in range(n_variables)])
df1.to_csv('train features')
df2=pd.DataFrame(train_labels)
df2.to_csv('train labels')

In [ ]:
test_features=np.reshape(test_features,(1000,n_variables))
df3=pd.DataFrame(test_features,columns = ['feature {}'.format(i) for i in range(n_variables)])
df3.to_csv('test features')
df4=pd.DataFrame(test_labels)
df4.to_csv('test labels')

In [ ]:
from google.colab import files
files.download('train features')
files.download('train labels')
files.download('test features')
files.download('test labels')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

b) Une fois les données stockées, une simple lecture me permet de les charger

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/train features',index_col=0)

In [ ]:
df3=pd.read_csv('/content/drive/MyDrive/test features',index_col=0)

In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/train labels',index_col=0)

In [ ]:
df4=pd.read_csv('/content/drive/MyDrive/test labels',index_col=0)

7.   Encoder les étiquettes



In [ ]:
def label_encoder (Y) : 
  encoded_labels = np.array([list(Y.T[i]).index(1) for i in range (Y.shape[0])])
  return encoded_labels

In [ ]:
y_train=label_encoder(df2)
y_test=label_encoder(df4)

 8.   Sélection des variables



a) Test d'homogéniété

In [ ]:
test=stats.normaltest(df1, axis=0, nan_policy='propagate')
score,p_value=test
result=[x<=0.05 for x in p_value]
# l'hypothèse de normalité peut être rejetée si p_value<=0.05
result.count(True)
# les échantillons ne suivent pas une loi normale ! 
#Pour cela j'ai opté pour une version non paramétrique du test ANOVA.

In [ ]:
# j'ai adapté le test kruskal pour l'utiliser avec SelectPercentile de Sklearn
def kruskal(X,y):
 X=pd.DataFrame(X)
 KruskalResult=[[],[]]
 i=0
 for c in X.columns:
   k=[[X[c][i] for i in range(10000) if y[i]==0],
     [X[c][i] for i in range(10000) if y[i]==1],
     [X[c][i] for i in range(10000) if y[i]==2],
      [X[c][i] for i in range(10000) if y[i]==3],
      [X[c][i] for i in range(10000) if y[i]==4],
     [X[c][i] for i in range(10000) if y[i]==5],
     [X[c][i] for i in range(10000) if y[i]==6],
     [X[c][i] for i in range(10000) if y[i]==7],
     [X[c][i] for i in range(10000) if y[i]==8],
     [X[c][i] for i in range(10000) if y[i]==9]]
   if k[0]==k[1]==k[2]==k[3]==k[4]==k[5]==k[6]==k[7]==k[8]==k[9]:
       KruskalResult[0].append(None)
       KruskalResult[0].append(None)
   else:
    KruskalResult[0].append(stats.kruskal(k[0],k[1],k[2],k[3],k[4],k[5],
                                          k[6],k[7],k[8],k[9])[0])
    KruskalResult[1].append(stats.kruskal(k[0],k[1],k[2],k[3],k[4],k[5],
                                          k[6],k[7],k[8],k[9])[1])
 return(KruskalResult)

b) Choisir le pourcentage de variables à garder

In [ ]:
# la standardisation est une étape primordiale
scaler=StandardScaler()

In [ ]:
def best_percentile_kernel(kernel):
  L=[]
  for p in tqdm([100,90,80,90,70,60,50,40,30,20,10,1]):
     selecteur=SelectPercentile(kruskal,percentile=p)
     selecteur.fit(df1,y_train)
     x_train=df1.T[selecteur.get_support()].T
     x_train=scaler.fit_transform(x_train)
     if kernel=='poly':
      model=SVC(decision_function_shape='ovo',kernel=kernel,gamma='scale',
                degree=2,C=1)
     elif kernel=='rbf':
      model=SVC(decision_function_shape='ovo',kernel=kernel,gamma='scale',C=1)
     else:
       model=SVC(decision_function_shape='ovo',kernel=kernel,C=1)
     L.append(cross_val_score(model,x_train, y_train, 
                              cv=10,verbose=10).mean()*100)
  return(L)

In [ ]:
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_kernel('linear'),
         label='noyau linéaire et c=1')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_kernel('rbf'),
         label='noyau RBF, gamma="scale" et c=1')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_kernel('poly'),
         label='noyau polynomial, gamma="scale", degré=2 et c=1')
plt.xlabel('percentile')
plt.ylabel('score (%)')
plt.title('Evolution du score en fonction du percentile')
plt.legend()
plt.show()

In [ ]:
def best_percentile_C(c):
  L=[]
  for p in tqdm([100,90,80,70,60,50,40,30,20,10,1]):
     selecteur=SelectPercentile(kruskal,percentile=p)
     selecteur.fit(df1,y_train)
     x_train=df1.T[selecteur.get_support()].T
     model=SVC(decision_function_shape='ovo',kernel='poly',gamma='scale',coef0=1,
               degree=2,C=c)
     L.append(cross_val_score(model,x_train, y_train, cv=10,
                              verbose=10).mean()*100))
  return(L)

In [ ]:
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_C(1),
         label='C=1')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_C(10),
         label='C=10')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_C(100),
         label='C=100')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_C(1000),
         label='C=1000')
plt.xlabel('percentile')
plt.ylabel('score (%)')
plt.title('Evolution du score en fonction du percentile gamma="scale",degré=2')
plt.legend()
plt.show()

In [ ]:
def best_percentile_gamma(gamma):
  L=[]
  for p in tqdm([100,90,80,70,60,50,40,30,20,10,1]):
     selecteur=SelectPercentile(kruskal,percentile=p)
     selecteur.fit(df1,y_train)
     x_train=df1.T[selecteur.get_support()].T
     x_train=scaler.fit_transform(x_train)
     model=SVC(decision_function_shape='ovo',kernel='poly',gamma=gamma,coef0=1,
               degree=2,C=1)
     L.append(cross_val_score(model,x_train, y_train, cv=10,
                              verbose=10).mean()*100)
  return(L)  

In [ ]:
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_gamma('scale'),
         label="Gamma='scale'")
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_gamma('auto'),
         label="Gamma='auto'")
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_gamma(1e-4),
         label='Gamma=1e-4')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_gamma(1e-2),
         label='Gamma=1e-2')
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_gamma(0.1),
         label='Gamma=0.1')
plt.xlabel('percentile')
plt.ylabel('score (%)')
plt.title('Evolution du score en fonction du percentile C=1, coef0=1 et degré=2')
plt.legend()
plt.show()

In [ ]:
def best_percentile_degree(degree):
  L=[]
  for p in tqdm([100,90,80,70,60,50,40,30,20,10,1]):
     selecteur=SelectPercentile(kruskal,percentile=p)
     selecteur.fit(df1,y_train)
     x_train=df1.T[selecteur.get_support()].T
     x_train=scaler.fit_transform(x_train)
     model=SVC(decision_function_shape='ovo',kernel='poly',gamma='scale',coef0=0,
               degree=degree,C=1)
     model.fit(x_train,y_train)
     L.append(cross_val_score(model,x_train, y_train, 
                              cv=10,verbose=10).mean()*100)
  return(L)

In [ ]:
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_degree(2),
         label="degré=2")
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_degree(3),
         label="degré=3")
plt.plot([100,90,80,70,60,50,40,30,20,10,1],best_percentile_degree(4),
         label="degré=4")
plt.xlabel('percentile')
plt.ylabel('score (%)')
plt.title("Evolution du score en fonction du percentile gamma='scale' et C=1")
plt.legend()
plt.show()

10. Ajuster les paramètres du modèle




a) Noyau linéaire

In [ ]:
selecteur=SelectPercentile(kruskal,percentile=80)
selecteur.fit(df1,y_train)
x_train=df1.T[selecteur.get_support()].T
x_train=scaler.fit_transform(x_train)

In [ ]:
parameters = {'C':[1e-5,1e-4,0.001,0.01,0.1,1,10,100,1000]}
clf = GridSearchCV(SVC(decision_function_shape='ovo',kernel='linear'),parameters,
                   verbose=10)
clf.fit(x_train,y_train)
clf.best_params_
clf.best_score_

b) Noyau RBF

In [ ]:
selecteur=SelectPercentile(kruskal,percentile=60)
selecteur.fit(df1,y_train)
x_train=df1.T[selecteur.get_support()].T
x_train=scaler.fit_transform(x_train)

In [ ]:
parameters = {'C':[0.1,1,10,100,1000],'gamma':[1e-7,1e-6,'auto','scale']}
clf = GridSearchCV(SVC(decision_function_shape='ovo',kernel='rbf'),parameters,
                   cv=10,verbose=10)
clf.fit(x_train,y_train)
clf.best_params_

12.   Évaluation des performances du meilleur modèle




In [ ]:
selecteur=SelectPercentile(kruskal,percentile=60)
selecteur.fit(df1,y_train)
x_train=df1.T[selecteur.get_support()].T
x_train=scaler.fit_transform(x_train)
x_test=df3.T[selecteur.get_support()].T
x_test=scaler.transform(x_test)

In [ ]:
model=SVC(decision_function_shape='ovo',kernel='rbf',C=100,gamma=1e-6)
model.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import classification_report
y_fit=model.predict(x_test)
print(classification_report(y_test, y_fit))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_fit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=class_names.keys(),yticklabels=class_names.keys())
plt.xlabel('true label')
plt.ylabel('predicted label');

13. Enregistrer le modèle

In [ ]:
# enregistrer le modèle
import pickle 
fichier='modèle_RBF_Gamma=1e-6_C=100.sav'
pickle.dump(model,open(fichier,'wb'))

In [ ]:
from google.colab import files
files.download('modèle_RBF_Gamma=1e-6_C=100.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

14. Tests sur terrain

In [ ]:
# importer le modèle 
import pickle 
model=pickle.load(open('/content/drive/MyDrive/modèle_RBF_Gamma=1e-6_C=100.sav',
                       'rb'))

In [ ]:
selecteur=SelectPercentile(kruskal,percentile=60)
selecteur.fit(df1,y_train)
scaler=StandardScaler()
selecteur.fit(df1,y_train)
x_train=df1.T[selecteur.get_support()].T
x_train=scaler.fit_transform(x_train)

In [ ]:
n_images=20
path='/content/drive/MyDrive/tests'
batch_size=1
target_size=(224,224)
class_names={0:'bacterial spot',1:'early blight',
             2:'late blight',3:'leaf mold',
             4:'septoria leaf spot',5:'spider mites',
             6:'target spot',7:'yellow leaf curl virus',
             8:'mosaic virus',9:'healthy'}
n_variables=vgg19.output_shape[1]*vgg19.output_shape[2]*vgg19.output_shape[3] 

In [ ]:
def prediction():
    images_dir = os.path.join(path)
    datagenerator = ImageDataGenerator()
    generator=datagenerator.flow_from_directory(images_dir,
   target_size=target_size,batch_size=batch_size,shuffle=False,class_mode=None)
    scaler=StandardScaler()
    x_train=df1.T[selecteur.get_support()].T
    x_train=scaler.fit_transform(x_train)
    i=0
    filenames=[]
    images=[]
    features = np.empty(shape=(n_images,vgg19.output_shape[1],
                               vgg19.output_shape[2],vgg19.output_shape[3]))
    for images_batch in tqdm(generator):
      images.append(images_batch[0].astype('uint'))
      preprocessed_batch=preprocess_input(images_batch)
      features[i*batch_size:(i+1)*batch_size]= vgg19.predict(preprocessed_batch)
      idx = generator.batch_index* generator.batch_size
      filenames.append(str(generator.filenames[idx : idx + generator.batch_size][0]).replace('images/','').replace('.jpg',''))
      i+= 1
      if i==n_images:
          break
    features=np.reshape(features,(n_images,n_variables))
    df=pd.DataFrame(features, columns = ['feature {}'.format(i) for i in range(n_variables)])
    x=df.T[selecteur.get_support()].T
    x=scaler.transform(x)
    prediction=model.predict(x)
    predicted_classes=[class_names[x] for x in prediction]
    return(images,filenames,predicted_classes)

In [ ]:
images,filenames,predicted_classes=prediction()
def Show(i):
  plt.figure(figsize=(5,5))
  plt.imshow(images[i])
  plt.title('predicted :'+predicted_classes[i]+'\n'+'True :'+filenames[i])
  plt.axis('off')